In [ ]:
import pandas as pd
import numpy as np
print(np.__version__)
print(pd.__version__)

In [ ]:
import os
os.getcwd()
# os.chdir(r'')

### Reading CSV files

In [ ]:
data = pd.read_csv(r"./data/students.txt")

In [ ]:
data.head(10)

how to change seperator, skip 2 lines at start and end of file?

In [ ]:
pd.read_csv?

In [ ]:
data = pd.read_csv(r"./data/students.txt",
                   sep='|',
                   header=2,
                   skipfooter=2)

How to include column(s) as my index?

In [ ]:
data

In [ ]:
data = pd.read_csv(r"./data/students.txt",
                   sep='|',
                   header=2,
                   skipfooter=2,
                   index_col=0)

In [ ]:
data

In [ ]:
data = pd.read_csv(r"./data/students.txt",
                   sep='|',
                   header=2,
                   skipfooter=2,
                   index_col=['first_name','last_name'])

In [ ]:
data

In [ ]:
data = pd.read_csv(r"./data/students.txt",
                   sep='|',
                   header=2,
                   skipfooter=2,
                   index_col='date_of_birth')

In [ ]:
data

### Reading excel files - sample data from Fred
Sheet1. RRPONTSYD

Sheet2. IOER

In [ ]:
xls = pd.read_excel(io=r'./data/fred.xls',
                    sheet_name = None)

In [ ]:
xls

In [ ]:
xls['RRPONTSYD'].head(11)

In [ ]:
RRPONTSYD = pd.read_excel(io=r'./data/fred.xls',
                          sheet_name = 'RRPONTSYD',
                         header=10,
                         index_col='observation_date')
IOER = pd.read_excel(io=r'./data/fred.xls',
                          sheet_name = 'IOER',
                         header=0,
                        index_col=0)

In [ ]:
RRPONTSYD.head(5)

In [ ]:
IOER.head(5)

How to plot 2 time series with 2 axes? 

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots()
ax.plot(IOER, color="red")
ax.set_xlabel("Date",fontsize=9)
ax.set_ylabel("O/N RR by Fed",color="red",fontsize=9)

ax2=ax.twinx()
ax2.plot(RRPONTSYD,color="blue")
ax2.set_ylabel("IOER",color="blue",fontsize=9)
plt.show()

### Selecting rows where a column satisfy some conditions

In [ ]:
A = (data.gender == 'F')
A

why this works when DataFrame is column oriented?

In [ ]:
type(A)

In [ ]:
A.values

In [ ]:
A.index

Because A is a Series, Series is like a dictionary
Map contains the exact same row indexes with a 1-1 mapping to a boolean value i.e. to include the row?

In [ ]:
data[A]

In [ ]:
data[(data.gender == 'F') & (data.ethnicity == 'White')]

### How to format my index to datetime format?

In [ ]:
data.index

In [ ]:
data.index.values

In [ ]:
type(data.index.values)

In [ ]:
data.index.values

how to find all methods of an object?

In [ ]:
[ i for i in dir(data.index) if i[0] != '_']

Being aware of object type means you know what methods that object has

In [ ]:
type(data.index.str)

In [ ]:
A = data.index.str.contains('/')
A

In [ ]:
data[A]

In [ ]:
data[~A]

how to drop rows from dataframe?

In [ ]:
try:
    data = data.drop(data[A].index,axis=0)
except:
    pass

how about drop columns?

In [ ]:
data.drop(['status','ethnicity'], axis=1)

In [ ]:
data.index = pd.to_datetime(data.index, format='%b-%y')
data.index

In [ ]:
data

How to sort by index?

In [ ]:
data.sort_index()

In [ ]:
data.sort_index(ascending=False)

### Stacking 2 dataframes

create sample dataframes

In [ ]:
t1, t2 = data[data.entry_age < 18.5], data[data.entry_age >= 18.5]

In [ ]:
t1.describe

In [ ]:
t2.shape

**with same column names**

In [ ]:
pd.concat([t1,t2])

In [ ]:
pd.concat([t1,t2]) is data

memory address of an object

In [ ]:
id(data)

In [ ]:
id(pd.concat([t1,t2]))

**with same row index**

how to create a new dataframe without reading?

In [ ]:
newColumns = pd.DataFrame({"colA":[1,2,3,4,5],
                          "colB":['test']*5},
                         index = data.index)

In [ ]:
newColumns

In [ ]:
data = pd.concat([newColumns, data, ], axis=1)
data

### Filtering columns

this works because?

In [ ]:
data[['colA','colB']]

In [ ]:
data.loc[:,['colA','colB']]

In [ ]:
data.iloc[:,0:2]

### Storing a sparse table 
Keep primary index, change column names to a column itself

In [ ]:
sparseMat = pd.DataFrame({"colA":[np.nan,'*','*',np.nan,np.nan],
                          "colB":[np.nan,np.nan,np.nan,'*','*']},
                         index = [1,2,3,4,5])

In [ ]:
sparseMat

In [ ]:
sparseMatIdx = pd.melt(sparseMat)
sparseMatIdx

In [ ]:
sparseMatIdx.dropna(axis=0)

Drop rows where **all** one column value is NaN

In [ ]:
sparseMat.dropna(axis=0,how='all')

Drop rows where **any** one column value is NaN

In [ ]:
sparseMat.dropna(axis=0,how='any')

In [ ]:
sparseMat['colA'] == np.nan

why don't work? cause np.nan is an instance of an object, not a static object

In [ ]:
id(np.nan)

In [ ]:
[id(i) for i in sparseMat['colA']]

In [ ]:
sparseMat[sparseMat['colA'].notna()]

pd.Dataframe.notna**??** returns its inline code documentation

In [ ]:
print(sparseMat['colA'].notna.__doc__)

### Aggregation

now, lets load a larger cross-sectional dataset from

https://data.gov.sg/dataset/resale-flat-prices

In [ ]:
dtypes = {'month': 'str',
          'town': 'category',
          'flat_type': 'category',
          'block': 'str',
          'street_name':'category',
          'storey_range':'category',
          'floor_area':'int',
          'flat_model':'category',
          'lease_commencement_date':'int',
          'remaining_lease':'str',
          'resale_price':'float'}

In [ ]:
resale_px = pd.read_excel(io=r'./data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.xlsx',
                          sheet_name = 0,
                         header=0,
                         index_col=None,
                         dtype = dtypes)

In [ ]:
resale_px.head()

In [ ]:
resale_px.shape

In [ ]:
resale_px.dtypes

In [ ]:
resale_px.flat_type.values

In [ ]:
print("categorical column [flat_type] used {:.2f}kB".format(resale_px.flat_type.memory_usage()/1024))

In [ ]:
print("string column [remaining_lease] used {:.2f}kB".format(resale_px.remaining_lease.memory_usage()/1024))

Group by 2 columns

In [ ]:
resale_px.groupby(by=['town','flat_type'])

In [ ]:
resale_px.groupby(by=['town','flat_type'], dropna=True)['resale_price']

In [ ]:
resale_px.groupby(by=['town','flat_type'], dropna=True)[['resale_price']]

In [ ]:
p1 = resale_px.groupby(by=['town','flat_type'], dropna=True)[['resale_price']].median()
p1

In [ ]:
p1.index

In [ ]:
p1.loc['TAMPINES']

In [ ]:
p1.loc[('TAMPINES','5 ROOM')]

In [ ]:
rm = p1.xs('5 ROOM', level="flat_type")
rm

In [ ]:
rm.plot.bar()

# PIVOT  TABLES

pd.Dataframe.pivot do not support aggregation

In [ ]:
i = ['town','flat_type']
c = []
v = "resale_price"
resale_px.pivot(index=i,
                columns=c,
                values=v)

pd.pivot_table(DataFrame) allow a range of aggfunc

In [ ]:
# pandas.pivot_table(data, 
#                    values=None, index=None, columns=None, 
#                    aggfunc='mean',
#                    fill_value=None,
#                    margins=False,
#                    dropna=True,
#                    margins_name='All',
#                    observed=False,
#                    sort=True)

i = ['town','flat_type']
c = []
v = "resale_price"
pd.pivot_table(data=resale_px,
               values=v,
               index=i,
               columns=c,
               aggfunc=np.median)

In [ ]:
i = ['street_name','flat_type']
c = ['storey_range']
v = "resale_price"
p2 = pd.pivot_table(data=resale_px,
               values=v,
               index=i,
               columns=c,
               aggfunc=np.median)
p2

In [ ]:
p2.index.get_level_values('street_name')

In [ ]:
p2.iloc[p2.index.get_level_values('street_name').str.contains("SIMEI"), 0:4]